In [1]:
from tenacity import retry, stop_after_attempt 
import requests
from bs4 import BeautifulSoup
import pandas as pd 

base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

@retry(stop=stop_after_attempt(3))
def get_html(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    return soup

all_data = []
max_page = 40

for page in range(1, max_page + 1):
    
    url = base_url.format(page)
    
    soup = get_html(url)
    
    items = soup.find_all("div",{"class":"cassetteitem"})
    print("page", page, "items", len(items))
    
    for item in items:
        stations = item.findAll("div",{"class":"cassetteitem_detail-text"})
        
        for station in stations:
            base_data = {}
            
            base_data["名称"] = item.find("div", {"class":"cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリ"] = item.find("div", {"class":"cassetteitem_content-label"}).getText().strip()
            base_data["住所"] = item.find("li", {"class":"cassetteitem_detail-col1"}).getText().strip()
            base_data["最寄り駅"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()
        
            tbodys = item.find("table", {"class":"cassetteitem_other"}).findAll("tbody")
            
            for tbody in tbodys:
                data = base_data.copy()
                
                data["階数"] = tbody.findAll("td")[2].getText().strip()
    
                data["賃料"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()
                
                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()
                
                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["専有面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()
            
                all_data.append(data)  

# convert to dataframe
df = pd.DataFrame(all_data)
df.to_csv("suumo01.csv", encoding="utf-8")


page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30
page 6 items 30
page 7 items 30
page 8 items 30
page 9 items 30
page 10 items 30
page 11 items 30
page 12 items 30
page 13 items 30
page 14 items 30
page 15 items 30
page 16 items 30
page 17 items 30
page 18 items 30
page 19 items 30
page 20 items 30
page 21 items 30
page 22 items 30
page 23 items 30
page 24 items 30
page 25 items 30
page 26 items 30
page 27 items 30
page 28 items 30
page 29 items 30
page 30 items 30
page 31 items 30
page 32 items 30
page 33 items 6
page 34 items 0
page 35 items 0
page 36 items 0
page 37 items 0
page 38 items 0
page 39 items 0
page 40 items 0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7065 entries, 0 to 7064
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   名称      7065 non-null   object
 1   カテゴリ    7065 non-null   object
 2   住所      7065 non-null   object
 3   最寄り駅    7065 non-null   object
 4   築年数     7065 non-null   object
 5   構造      7065 non-null   object
 6   階数      7065 non-null   object
 7   賃料      7065 non-null   object
 8   管理費     7065 non-null   object
 9   敷金      7065 non-null   object
 10  礼金      7065 non-null   object
 11  間取り     7065 non-null   object
 12  専有面積    7065 non-null   object
dtypes: object(13)
memory usage: 717.7+ KB


In [4]:
df.duplicated(subset=['階数','賃料','管理費','間取り','専有面積'])

0       False
1        True
2        True
3       False
4       False
        ...  
7060     True
7061     True
7062    False
7063     True
7064     True
Length: 7065, dtype: bool

In [6]:
df.drop_duplicates(subset=['階数','賃料','管理費','間取り','専有面積'])
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2
1,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ銀座線/末広町駅 歩5分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2
2,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,都営新宿線/岩本町駅 歩7分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,4階,14.4万円,10000円,14.4万円,-,1DK,29.02m2
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,8階,27.7万円,13000円,27.7万円,-,2LDK,54.42m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7060,ハミルトンプレイス,賃貸マンション,東京都千代田区三番町,東京メトロ東西線/九段下駅 歩9分,築18年,9階建,4階,14.4万円,10000円,14.4万円,14.4万円,ワンルーム,33.5m2
7061,ハミルトンプレイス,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築18年,9階建,4階,14.4万円,10000円,14.4万円,14.4万円,ワンルーム,33.5m2
7062,ＪＲ山手線 神田駅 4階建 築45年,賃貸マンション,東京都千代田区東神田１,ＪＲ山手線/神田駅 歩10分,築45年,4階建,3階,5万円,11000円,-,-,ワンルーム,5m2
7063,ＪＲ山手線 神田駅 4階建 築45年,賃貸マンション,東京都千代田区東神田１,都営浅草線/東日本橋駅 歩5分,築45年,4階建,3階,5万円,11000円,-,-,ワンルーム,5m2


In [7]:
df.drop_duplicates(subset=['階数','賃料','管理費','間取り','専有面積'], inplace=True)
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,4階,14.4万円,10000円,14.4万円,-,1DK,29.02m2
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,8階,27.7万円,13000円,27.7万円,-,2LDK,54.42m2
5,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,11階,28.3万円,13000円,28.3万円,-,2LDK,54.42m2
12,KWレジデンス麹町,賃貸マンション,東京都千代田区麹町３,東京メトロ有楽町線/麹町駅 歩4分,築19年,地下1地上15階建,6階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,コーポ麹町,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,14.5万円,-,14.5万円,14.5万円,1DK,37.72m2
7044,東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,15万円,-,15万円,15万円,1DK,37.72m2
7047,ＪＲ山手線 秋葉原駅 5階建 築45年,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,築45年,5階建,5階,6.8万円,5000円,6.8万円,6.8万円,ワンルーム,15.47m2
7053,ＪＲ総武線 飯田橋駅 14階建 築11年,賃貸マンション,東京都千代田区神田神保町１,ＪＲ総武線/飯田橋駅 歩17分,築11年,14階建,7階,12.1万円,10000円,12.1万円,12.1万円,1K,25.06m2


In [8]:
df.to_csv("suumo05.csv", encoding="utf-8")

In [11]:
df.duplicated(subset=['階数','賃料','管理費','間取り','専有面積'])

0       False
3       False
4       False
5       False
12      False
        ...  
7041    False
7044    False
7047    False
7053    False
7062    False
Length: 1341, dtype: bool

In [28]:
print(df.duplicated().sum())

580


In [30]:
print((~df.duplicated()).sum())
print(df.duplicated().value_counts())

6365
False    6365
True      580
dtype: int64


In [26]:
df2 = df[~df.duplicated()]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6365 entries, 0 to 6944
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   名称      6365 non-null   object
 1   カテゴリ    6365 non-null   object
 2   住所      6365 non-null   object
 3   最寄り駅    6365 non-null   object
 4   築年数     6365 non-null   object
 5   構造      6365 non-null   object
 6   階数      6365 non-null   object
 7   賃料      6365 non-null   object
 8   管理費     6365 non-null   object
 9   敷金      6365 non-null   object
 10  礼金      6365 non-null   object
 11  間取り     6365 non-null   object
 12  専有面積    6365 non-null   object
dtypes: object(13)
memory usage: 696.2+ KB


In [31]:
df2.duplicated(subset=['階数','賃料','管理費','間取り','専有面積'])

0       False
1        True
2        True
3       False
4       False
        ...  
6940     True
6941     True
6942    False
6943     True
6944     True
Length: 6365, dtype: bool